In [1]:
! pip install transformers datasets
! pip install transformers datasets evaluate
! pip install transformers[torch]
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
import time
import torch
import math
from tqdm import tqdm
import wandb
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
notebook_login()

In [ ]:
print('hello')

hello


In [5]:

dataset = load_dataset("gpt3mix/sst2")

Generating train split:   0%|          | 0/6920 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
dataset["validation"][0]

{'text': "It 's a lovely film with lovely performances by Buy and Accorsi .",
 'label': 0}

In [7]:
test=dataset['test']
#test['label']

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_review = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

accuracy = evaluate.load("accuracy")

In [ ]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "POSITIVE", 1: "NEGATIVE"}
label2id = {"POSITIVE": 0, "NEGATIVE": 1}

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomTrainer(Trainer):
    def _inner_training_loop(
        self, batch_size=None, args=None, resume_from_checkpoint=None, trial=None, ignore_keys_for_eval=None
    ):
        wandb.init(
        # set the wandb project where this run will be logged
        project="my-awesome-project",

        # track hyperparameters and run metadata
        config={
        "learning_rate": args.learning_rate,
        "architecture": "NN",
        "dataset": "SST2",
        "epochs": args.num_train_epochs,
        }
        )
        number_of_epochs = args.num_train_epochs
        start = time.time()
        train_loss =[]
        train_acc =[]
        eval_acc =[]
        criterion = torch.nn.BCEWithLogitsLoss ().to(device) # to make all the computations happen through cuda
        self.optimizer = torch.optim.Adam(model.parameters(),lr=args.learning_rate )
        self.scheduler = torch.optim.lr_scheduler.StepLR (self.optimizer,1,gamma=0.9)
        train_dataloader = self.get_train_dataloader()
        eval_dataloader = self.get_eval_dataloader()
        max_steps = math . ceil(args.num_train_epochs * len ( train_dataloader ))
        for epoch in range(number_of_epochs):
          train_loss_per_epoch = 0
          train_acc_per_epoch = 0
          with tqdm (train_dataloader,unit=" batch ") as training_epoch :
            training_epoch.set_description(f"Training Epoch { epoch }")
            for step,inputs in enumerate(training_epoch):
              inputs = inputs.to (device)
              #return(inputs)
              labels = inputs['labels'].float()
              # forward pass
              self.optimizer.zero_grad ()
              output = model(**inputs)# TODO Implement by yourself
              # get the loss
              loss = criterion(output[1][:,1],labels) # TODO Implement by yourself
              curr_acc=(output ['logits'].argmax(1)==labels).sum().item()
              # print(f'\tTrain Loss per loop: {loss:.3f} | Train Acc per loop: {curr_acc*100:.2f}%') ---- CHECK THIS
              train_loss_per_epoch += loss.item ()
              # calculate gradients
              loss.backward ()
              # update weights
              self.optimizer.step()
              train_acc_per_epoch += (output ['logits'].argmax(1)==labels).sum().item()
          # adjust the learning rate
          self.scheduler.step ()
          train_loss_per_epoch /= len ( train_dataloader )
          train_acc_per_epoch /= (len ( train_dataloader )* batch_size )
          eval_loss_per_epoch = 0
          eval_acc_per_epoch = 0
          with tqdm ( eval_dataloader , unit =" batch ") as eval_epoch :
            eval_epoch . set_description (f" Evaluation Epoch { epoch }")
            for step,inputs in enumerate(eval_epoch):
              inputs = inputs.to (device)
              labels = inputs['labels'].float()
              # forward pass
              output = model(**inputs)
              loss = criterion(output[1][:,1],labels)
              eval_loss_per_epoch += loss.item ()
              eval_acc_per_epoch += (output['logits'].argmax(1)==labels).sum().item()
            # ... TODO Implement by yourself
          eval_loss_per_epoch /= (len ( eval_dataloader ))
          eval_acc_per_epoch /= ( len ( eval_dataloader )* batch_size )
          print (f'\n\t Train Loss:{train_loss_per_epoch:.3f} | Train Acc : {train_acc_per_epoch * 100 :.2f}% ') # These give the accuracy after each epoch ( epoch is traning through entire training set)
          print (f'\t Eval Loss:{eval_loss_per_epoch:.3f} | Eval Acc : {eval_acc_per_epoch * 100 :.2f}%')
          wandb.log({"val_acc": eval_acc_per_epoch, "loss": train_loss_per_epoch,"train_acc":train_acc_per_epoch})
        print (f'Time :{(time.time()-start)/60:.3f} minutes ')
        wandb.finish()

In [ ]:
training_args = TrainingArguments(
    output_dir="homework_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

cust_trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_review["train"],
    eval_dataset=tokenized_review["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
cust_trainer.train()

wandb: Currently logged in as: thono001 (stonekraken). Use `wandb login --relogin` to force relogin


 Evaluation Epoch 0: 100%|██████████| 55/55 [00:01<00:00, 47.77 batch /s]



	 Train Loss:0.346 | Train Acc : 84.97% 
	 Eval Loss:0.304 | Eval Acc : 86.93%


 Evaluation Epoch 1: 100%|██████████| 55/55 [00:01<00:00, 45.79 batch /s]



	 Train Loss:0.124 | Train Acc : 95.76% 
	 Eval Loss:0.318 | Eval Acc : 87.16%


...

 Evaluation Epoch 13: 100%|██████████| 55/55 [00:01<00:00, 44.93 batch /s]



	 Train Loss:0.000 | Train Acc : 99.88% 
	 Eval Loss:0.671 | Eval Acc : 88.52%


 Evaluation Epoch 14: 100%|██████████| 55/55 [00:01<00:00, 43.29 batch /s]



	 Train Loss:0.000 | Train Acc : 99.88% 
	 Eval Loss:0.686 | Eval Acc : 88.52%
Time :8.898 minutes 


loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▇████████████
val_acc,▃▃▆▆▁█▅▇▆▆▆▆▆▆▆
loss,8e-05
train_acc,0.99885
val_acc,0.88523


In [ ]:
cust_trainer.push_to_hub()

events.out.tfevents.1707692461.8b52cd0b9ef0.732.1:   0%|          | 0.00/604 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Stonekraken/homework_1/commit/02602f6aa80f1c71b88921e91f68b040f41f6b29', commit_message='End of training', commit_description='', oid='02602f6aa80f1c71b88921e91f68b040f41f6b29', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
evaluation_results = cust_trainer.evaluate(tokenized_review["test"])
evaluation_results

{'eval_loss': 0.6040809750556946,
 'eval_accuracy': 0.8978583196046128,
 'eval_runtime': 2.6188,
 'eval_samples_per_second': 695.347,
 'eval_steps_per_second': 43.531}

In [ ]:
evaluation_results

{'eval_loss': 0.6040809750556946,
 'eval_accuracy': 0.8978583196046128,
 'eval_runtime': 2.6188,
 'eval_samples_per_second': 695.347,
 'eval_steps_per_second': 43.531}

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("Stonekraken/homework_1")
test_results = cust_trainer.evaluate(tokenized_review["test"])
test_results

In [ ]:
model.config.hidden_size

768

In [ ]:
model.config.dropout

0.1

In [ ]:
model.config

DistilBertConfig {
  "_name_or_path": "Stonekraken/homework_1",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "POSITIVE",
    "1": "NEGATIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 1,
    "POSITIVE": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "vocab_size": 30522
}

In [ ]:
!nvidia-smi

Sun Feb 11 23:21:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0              29W /  70W |   1921MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Analysis of model:

In [ ]:
#1.The problem statement is to identify the sentiment behind the movie review provided by the audience,Sentiment behind the review can either be positive or negative.
#In order to solve this problem I am using the pre trained instance of distilbert-base-uncased and fine tuning the model to analyze movie review data.
# In the reference paper Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank the author

#2.In order to run this model I have used the T4 GPU in google colab to effictively utilize the parallel processing capabilites of the GPU and reduce the computation time.

#3.We can observe a downward trend in the loss fucntion as expected, We can also observe an increase in validation and training accuray as well indicating that the model is performing correctly.

#4.In this experiment I have used accuracy as the key metric to determine the performance of my model, this is primarly because accuracy is used as the model bench mark in the learderboard provided,
# hence a comparison can be established between the current board and my model.

#5 Based on the test set we can observea an accuracy of 89.78%. The orginal paper reported an accuracy score of 85.4% using Recursive Neural Tensor Network. The increase in accuracy of this model can be
# expalined due to the fact that distilbert-base-uncased has been selected and the inital pretrained model.

#6 Training time 8.898 minutes , for the test data we can observe an inference time of 2.6188s.

#7 Hyperparameter
# Epochs:20, Learning rate:2e-5, Hidden size of model: 768 ,dropout rate:0.1, model type:distilbert, activation funciton:gelu

In [8]:
#8 Based on the ten examples with incorrect precition we can observe that the incorrectly predicted some of statement are abmigous without any clear positve or negative sentiment associated with it.
# Human annotators also seem to have split decision on these statements.
# Some of the incorrectly predicted statement have double negation which might cause our model to interpret it incorrectly.
output_df=pd.DataFrame()
model_name='Stonekraken/homework_1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
format={'Text':[],'Confidence score for Postive':[],'Confidence score for Negative':[],'Prediction':[],'Ground Truth':[]}
Analysis=pd.DataFrame(format)
count=0
i=0
while count!=10:
  text=test['text'][i]
  inputs = tokenizer(text,return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  if predicted_class_id!=test['label'][i]:
    print(text)
    print('Confidence score:',torch.nn.functional.softmax(logits,dim=1))
    print('Predict:',model.config.id2label[predicted_class_id],"->Actual:",model.config.id2label[test['label'][i]])
    count+=1
  i+=1

If there 's a way to effectively teach kids about the dangers of drugs , I think it 's in projects like the -LRB- unfortunately R-rated -RRB- Paid .
Confidence score: tensor([[4.7487e-04, 9.9953e-01]])
Predict: NEGATIVE ->Actual: POSITIVE
Ms. Fulford-Wierzbicki is almost spooky in her sulky , calculating Lolita turn .
Confidence score: tensor([[6.1568e-04, 9.9938e-01]])
Predict: NEGATIVE ->Actual: POSITIVE
If this movie were a book , it would be a page-turner , you ca n't wait to see what happens next .
Confidence score: tensor([[0.0406, 0.9594]])
Predict: NEGATIVE ->Actual: POSITIVE
Absorbing and disturbing -- perhaps more disturbing than originally intended -- but a little clarity would have gone a long way .
Confidence score: tensor([[9.9978e-01, 2.1662e-04]])
Predict: POSITIVE ->Actual: NEGATIVE
But in Imax 3-D , the clichés disappear into the vertiginous perspectives opened up by the photography .
Confidence score: tensor([[0.0094, 0.9906]])
Predict: NEGATIVE ->Actual: POSITIVE
If

In [ ]:
#np.isnan(test['text']).sum()

In [9]:
count=0
i=0
format={'Confidence score for Postive':[],'Confidence score for Negative':[],'Prediction':[],'Ground Truth':[]}
Analysis=pd.DataFrame(format)
model_name='Stonekraken/homework_1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
while count!=10:
  text=test['text'][i]
  inputs = tokenizer(text,return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  if predicted_class_id!=test['label'][i]:
    array = torch.nn.functional.softmax(logits,dim=1).numpy()
    new_data={'Text':text,'Confidence score for Postive':[array[0][0]],'Confidence score for Negative':[array[0][1]],'Prediction':model.config.id2label[predicted_class_id],"Ground Truth":model.config.id2label[test['label'][i]]}
    new_row=pd.DataFrame(new_data)
    Analysis=pd.concat([Analysis, new_row], ignore_index=True)
    count+=1
  i+=1

In [17]:
Analysis.to_csv('Incorrect_preditions.csv')

In [ ]:
#9 Combining predictions from multiple sentiment analysis models can help mitigate errors and improve overall performance.
# Conducting thorough error analysis to understand the types of errors made by the model and iteratively refining the model based on insights gained from the analysis
# can lead to continuous improvement in sentiment analysis performance.

In [ ]:
#10 I felt the most challenging part of this assignment was finding out 10 mispredicted values and analysing them for improvements.